# Exercise 2

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import csv

In [15]:
base_url = 'https://arxiv.org'
cs_recent_url = '/list/cs/pastweek?skip=0&show=250'

In [16]:
headers = {
    'authority': 'curlconverter.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'accept-language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'cache-control': 'max-age=0',
    'if-modified-since': 'Fri, 15 Jul 2022 21:44:42 GMT',
    'if-none-match': 'W/"62d1dfca-3a58"',
    'referer': 'https://curlconverter.com/',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="102", "Microsoft Edge";v="102"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Linux"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30',
}

In [17]:
page = requests.get(base_url + cs_recent_url, headers=headers)

soup = BeautifulSoup(page.text, 'html.parser')

In [18]:

entries = soup.find_all('div', class_='meta')
dt_entries = soup.find_all('dt')


In [19]:
row=0;

In [20]:
with open("papers.csv", mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # Write the CSV header
    writer.writerow(['Title', 'Authors', 'Subjects', 'Abstract'])
    for i, (meta_entry, dt_entry) in enumerate(zip(entries, dt_entries)):
        if row >= 100:  
            break
        title = meta_entry.find('div', class_='list-title').get_text(strip=True)
        authors = meta_entry.find('div', class_='list-authors').get_text(strip=True)
        subjects = meta_entry.find('div', class_='list-subjects').get_text(strip=True)

 
        list_identifier_span = dt_entry.find('span', class_='list-identifier')
        abstract_a_tag = list_identifier_span.find('a', title='Abstract') if list_identifier_span else None
    
        if abstract_a_tag:
            abstract_link = base_url + abstract_a_tag['href']
            try:
                response = requests.get(abstract_link, headers=headers, timeout=3)
                response.raise_for_status()  
                
                abstract_soup = BeautifulSoup(response.content, 'html.parser')
                abstract = abstract_soup.find('blockquote', class_='abstract').get_text(strip=True).replace('Abstract:', '').strip()
                
                writer.writerow([title, authors, subjects, abstract])  
                row=row+1
                print(f"Processed entry {i+1}: {title}")
                
            except requests.exceptions.Timeout:
                print(f"Request for entry {i+1} timed out. Skipping.")
                continue  # Skip to the next iteration of the loop
            except requests.exceptions.HTTPError as e:
                print(f"HTTP error for entry {i+1}: {e}. Skipping.")
                continue  # Skip to the next iteration of the loop

            time.sleep(3)  
        else:
            print(f"No abstract link found for title: {title}")

Processed entry 1: Title:Point Could Mamba: Point Cloud Learning via State Space Model
Processed entry 2: Title:Mitigating Reversal Curse via Semantic-aware Permutation Training
Processed entry 3: Title:An Experimental Study of Low-Latency Video Streaming over 5G
Processed entry 4: Title:AtP*: An efficient and scalable method for localizing LLM behaviour to  components
Processed entry 5: Title:Neural Acceleration of Incomplete Cholesky Preconditioners
Processed entry 6: Title:Dialect prejudice predicts AI decisions about people's character,  employability, and criminality
Processed entry 7: Title:Happy Ending: An Empty Hexagon in Every Set of 30 Points
Processed entry 8: Title:Can Transformers Capture Spatial Relations between Objects?
Processed entry 9: Title:Cost-Effective Activity Control of Asymptomatic Carriers in Layered  Temporal Social Networks
Processed entry 10: Title:Few-Shot Relation Extraction with Hybrid Visual Evidence
Processed entry 11: Title:Subhomogeneous Deep Equili